<a href="https://colab.research.google.com/github/2003UJAN/Phising-Website/blob/main/url_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from urllib.parse import urlparse,parse_qs
import csv
from google.colab import files
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse


In [2]:
#loading the dataset
data = pd.read_csv("/Phishing_Legitimate_full.csv")

In [3]:
#data pre-processing
data = data.drop(data.columns[0], axis=1)
data = shuffle(data)
threshold = 0.9 * len(data)
data = data.dropna(thresh=threshold, axis=1)

In [4]:
first_row = data.iloc[0]

In [5]:
print(first_row)

NumDots                                3.000000
SubdomainLevel                         1.000000
PathLevel                              1.000000
UrlLength                             80.000000
NumDash                                0.000000
NumDashInHostname                      0.000000
AtSymbol                               0.000000
TildeSymbol                            0.000000
NumUnderscore                          0.000000
NumPercent                             0.000000
NumQueryComponents                     3.000000
NumAmpersand                           2.000000
NumHash                                0.000000
NumNumericChars                       10.000000
NoHttps                                1.000000
RandomString                           0.000000
IpAddress                              0.000000
DomainInSubdomains                     0.000000
DomainInPaths                          0.000000
HttpsInHostname                        0.000000
HostnameLength                        18

In [6]:
urls = [
    'https://www.nitk.ac.in/',
    'https://www.meity.gov.in/',
    'https://www.srmist.edu.in/'
]

In [11]:
features={}

In [19]:
def extract_features(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
     num_dots = parsed_url.hostname.count('.')
    subdomain_level = len(parsed_url.hostname.split('.')) - 2
    path_level = len(parsed_url.path.split('/')) - 1
    url_length = len(url)
    num_dash_in_hostname = any(char.isdigit() for char in parsed_url.hostname if char == '-')
    has_at_symbol = '@' in url
    has_tilde_symbol = '~' in url
    num_underscore = url.count('_')
    num_percent = url.count('%')
    num_hash = url.count('#')
    num_numeric_chars = sum(c.isdigit() for c in url)
    query_components = parse_qs(parsed_url.query)
    num_query_components = len(query_components)
    num_ampersand = 0
    for param in query_components:
        num_ampersand += len(query_components[param]) - 1
    no_https = not parsed_url.scheme == 'https'
    random_string = parsed_url.path
    ip_address = parsed_url.hostname if parsed_url.hostname.replace('.', '').isdigit() else None
    subdomains = parsed_url.hostname.split('.')[:-2] if ip_address is None else None
    path_segments = parsed_url.path.split('/')
    domain_in_paths = None
    for segment in path_segments:
        if segment.count('.') >= 1:
            domain_in_paths = segment.split('.')[0]
            break
    is_https = parsed_url.scheme == 'https'
    hostname_length = len(parsed_url.hostname)
    path_length = len(parsed_url.path)
    query_length = len(parsed_url.query)
    double_slash_in_path = '//' in parsed_url.path
    url_words = url.lower().split('/')
    num_sensitive_words = sum(word in sensitive_words for word in url_words)
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
    brand_name = 'YourBrandName'
        brand_in_url = brand_name.lower() in url.lower()
        brand_in_content = brand_name.lower() in soup.get_text().lower()
        total_links = len(soup.find_all('a'))
        ext_links = sum(1 for link in soup.find_all('a') if urlparse(link.get('href')).netloc != urlparse(url).netloc)
        pct_ext_hyperlinks = (ext_links / total_links) * 100 if total_links > 0 else 0
    total_resources = len(soup.find_all(['img', 'script', 'link']))
        ext_resources = sum(1 for res in soup.find_all(['img', 'script', 'link']) if urlparse(res.get('src', '')).netloc != urlparse(url).netloc)
        pct_ext_resources = (ext_resources / total_resources) * 100 if total_resources > 0 else 0
    favicon_link = soup.find('link', rel='icon')
        ext_favicon = False
        if favicon_link:
            favicon_url = favicon_link.get('href', '')
            ext_favicon = urlparse(favicon_url).netloc != urlparse(url).netloc

      response = requests.get(url)
    if response.status_code == 200:
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
      submit_info_to_email = bool(soup.find('form', attrs={'action': 'mailto:'}))

        iframe_or_frame = bool(soup.find_all(['iframe', 'frame']))

        missing_title = not bool(soup.find('title'))
        parsed_url = urlparse(url)
        subdomains = len(parsed_url.netloc.split('.'))

        url_length = len(url)
        scheme = parsed_url.scheme
        netloc = parsed_url.netloc
        path = parsed_url.path
        query_params = parse_qs(parsed_url.query)

    return {
        'Domain': domain,
        'NumDots': num_dots,
        'SubdomainLevel': subdomain_level,
        'PathLevel': path_level,
        'UrlLength': url_length,
        'NumDash': has_num_dash,
        'NumDashInHostname': num_dash_in_hostname,
        'AtSymbol': has_at_symbol,
        'TildeSymbol': has_tilde_symbol,
        'NumUnderscore': num_underscore,
        'NumPercent': num_percent,
        'NumQueryComponents': num_query_components,
        'NumAmpersand': num_ampersand,
        'NumHash': num_hash,
        'NumNumericChars': num_numeric_chars,
        'NoHttps': no_https,
        'RandomString': random_string,
        'IpAddress': ip_address,
        'DomainInSubdomains': subdomains,
        'DomainInPaths': domain_in_paths,
        'HttpsInHostname': is_https,
        'HostnameLength': hostname_length,
        'PathLength': path_length,
        'QueryLength': query_length,
        'DoubleSlashInPath': double_slash_in_path,
        'NumSensitiveWords': num_sensitive_words,
        'EmbeddedBrandName': brand_in_url or brand_in_content,
        'PctExtHyperlinks': pct_ext_hyperlinks,
        'PctExtResourceUrls': pct_ext_resources,
        'ExtFavicon': ext_favicon,
        'SubmitInfoToEmail': submit_info_to_email,
            'IframeOrFrame': iframe_or_frame,
            'MissingTitle': missing_title,
            'SubdomainLevelRT': subdomains,
            'UrlLengthRT': url_length
    }
    print(f"Scheme: {scheme}")
    print(f"Netloc: {netloc}")
    print(f"Path: {path}")
    print(f"Query Parameters: {query_params}")
sensitive_words = ['sensitive', 'blacklisted', 'confidential']
extracted_features = []
for url in urls:
    features = extract_features(url)
    extracted_features.append(features)

IndentationError: ignored

In [ ]:
df = pd.DataFrame(extracted_features)

In [ ]:
print(df)

In [ ]:
def save_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['Domain','DomainLength','NumSubdomains', 'Path', 'PathLength', 'NumPathSegments', 'NumDash','NumDashInHostname','AtSymbol',
                      'TildeSymbol','NumUnderscore','NumPercent','NumQueryComponents', 'NumAmpersand', 'NumHash','NumNumericChars','NoHttps',
                      'RandomString','IpAddress', 'DomainInSubdomains','DomainInPaths', 'HttpsInHostname','HostnameLength','PathLength'
                       'QueryLength',' DoubleSlashInPath',' NumSensitiveWords',' EmbeddedBrandName','PctExtHyperlinks',' PctExtResourceUrls',' ExtFavicon'
                      'InsecureForms','RelativeFormAction','ExtFormAction','AbnormalFormAction','PctNullSelfRedirectHyperlinks'
                      ,'FrequentDomainNameMismatch','FakeLinkInStatusBar','RightClickDisabled','PopUpWindow','SubmitInfoToEmail','IframeOrFrame'
                      ,'MissingTitle','ImagesOnlyInForm','SubdomainLevelRT','UrlLengthRT','PctExtResourceUrlsRT','AbnormalExtFormActionR'
                      ,'ExtMetaScriptLinkRT','PctExtNullSelfRedirectHyperlinksRT' ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [ ]:
csv_filename = 'url_features.csv'
save_to_csv(extracted_features, csv_filename)
print(f"Features saved to '{csv_filename}'")
files.download(csv_filename)